In [325]:
import re
from datetime import datetime as dt
import datetime
from collections import defaultdict
import yaml
from time import time
import plotly.graph_objects as go
import numpy as np

# Functions

In [326]:

def epoch_idx_finder(lines, pattern):
    idx = []

    for i, line in enumerate(lines):
        match = re.match(pattern, line)

        if match:
            idx.append(i)
        else:
            continue

    return idx


def nav_epoch_pattern(prn="A"):
    
    if len(prn) == 1:
        if prn in "A":
            epoch_pattern = "^[GERSC]\s*\d+ "
        elif prn in "GERSCJ":
            epoch_pattern = f"^{prn}\s*\d+ "

    elif len(prn) > 1 and prn[1] in "1234567890":
        epoch_pattern = f"^{prn[0]}\s*{prn[1:]} "

    else:
        raise ValueError

    return epoch_pattern


def num_lines_nav(idx, nav_lines):

    i = 1
    line = nav_lines[idx + i]

    while not re.match(r"^[GERSCJ]\s*\d+", line):
        i += 1
        try:
            line = nav_lines[idx + i]
        except IndexError:
            break

    return i

def nav_pattern_finder(schema, epoch):

    result = {}
    for i, (line, schema_line) in enumerate(zip(epoch, schema)):
        pattern = "".join(
            f"(?P<{f['name']}>{f['pattern']})" for f in schema_line["fields"]
        )
        match = re.match(pattern, line.strip())
        if not match:
            print(f"خط {i+1} match نشد ❌")
            continue

        for key, value in match.groupdict().items():
            # print(match.groupdict())
            # print(key, value)
            if key == "prn":
                result[key] = value
                continue

            if key in {"year", "month", "day", "hour", "minute"}:
                value = int(float(value))

                if key == "year" and value < 100:
                    value += 2000

            else:
                if "D" in value:
                    value = value.replace("D", "E")
                value = float(value)

            result[key] = value

    return result


def make_nav_dataframe(dataframe, epoch_items):

    prn, year, month, day, hour, minute, second = (
        epoch_items["prn"],
        epoch_items["year"],
        epoch_items["month"],
        epoch_items["day"],
        epoch_items["hour"],
        epoch_items["minute"],
        epoch_items["second"],
    )
    epoche_date = dt(year, month, day, hour, minute, int(second))
    dataframe[prn][epoche_date] = epoch_items
    return dataframe


def navigation_reader(address, prn="A"):
    with open(address, "r") as file:
        nav_lines = file.readlines()
        
    with open("hw3_test.yaml", "r") as file:
        schema = yaml.safe_load(file)["nav_block"]

    nav_data = defaultdict(dict)

    epoch_pattern = nav_epoch_pattern(prn)
    epoch_idx = epoch_idx_finder(nav_lines, epoch_pattern)

    for idx in epoch_idx:
        j = num_lines_nav(idx, nav_lines)
        full_epoch_lines = nav_lines[idx : idx + j]

        res = nav_pattern_finder(schema, full_epoch_lines)
        nav_data = make_nav_dataframe(nav_data, res)

    return nav_data

In [327]:
def calc_tk(te, toe):

    tk = te - toe
    if tk > 302400:
        tk -= 604800

    elif tk < -302400:
        tk += 604800

    return tk


def solve_kepler(M, e, tol=1e-12, max_iter=100):
    E = M  # حدس اولیه
    for _ in range(max_iter):
        f = E - e * np.sin(E) - M
        f_prime = 1 - e * np.cos(E)
        delta = f / f_prime
        E_new = E - delta
        if abs(delta) < tol:
            return E_new
        E = E_new
    raise RuntimeError("Kepler equation did not converge")


def calc_gps_time(dtime):
    gps_epoch = dt(1980, 1, 6, 0, 0, 0)
    delta = dtime - gps_epoch
    total_seconds = delta.total_seconds()
    gps_week = int(total_seconds // 604800)
    seconds_of_week = total_seconds % 604800

    return gps_week, seconds_of_week


def calc_gps_seconds(dtime):
    gps_epoch = dt(1980, 1, 6, 0, 0, 0)
    delta = dtime - gps_epoch
    return delta.total_seconds()


def eci_to_ecef(Xeci, Yeci, Zeci, tk, omega_e=7.2921151467e-5):

    theta = omega_e * tk

    Xecef = np.cos(theta) * Xeci + np.sin(theta) * Yeci
    Yecef = -np.sin(theta) * Xeci + np.cos(theta) * Yeci
    Zecef = Zeci

    return Xecef, Yecef, Zecef

# Main Part

In [ ]:
nav_data = navigation_reader('./GODS00USA_R_20240010000_01D_GN.rnx', 'G') # read navigation file
prn = "G07" # choose what gps to calculate

toe_list = np.array(list(nav_data[prn].keys()))
te_list = []
t_start = toe_list[0]
t_end = toe_list[-1]

current_time = t_start
while current_time <= t_end:
    te_list.append(current_time)
    current_time += datetime.timedelta(seconds=300)

mu, omega_e = 3.986005e14, 7.2921151467e-5
Xecef_list, Yecef_list, Zecef_list = [], [], []
Xeci_list, Yeci_list, Zeci_list = [], [], []
t = []

for te in te_list:
    t1 = time()

    
    """  
    here is the part i guss my problem is:
    i thin problem is whtitin tk or toe (time of ephemeris) """
    
    
    toe = toe_list[np.argmin(np.abs(te - toe_list))]
    
    """this part (toe = toe_list[np.argmin(np.abs(te - toe_list))]) 
    is trying to find and use nearest epoch for each time (te - time of emission)"""
    
    # toe = toe_list[3]
    
    """ when i use only one epoch to calculate satellite position i (e.g. toe = toe_list[3])
    the orbit is right . but this use is wrong from what i leared.
    """

    M0 = nav_data[prn][toe]["M0"]
    dn = nav_data[prn][toe]["dn"]
    a = (nav_data[prn][toe]["sqrtA"]) ** 2
    e = nav_data[prn][toe]["e"]
    omega = nav_data[prn][toe]["omega"]
    Cuc = nav_data[prn][toe]["Cuc"]
    Cus = nav_data[prn][toe]["Cus"]
    Crc = nav_data[prn][toe]["Crc"]
    Crs = nav_data[prn][toe]["Crs"]
    Cic = nav_data[prn][toe]["Cic"]
    Cis = nav_data[prn][toe]["Cis"]
    i0 = nav_data[prn][toe]["i0"]
    i_dot = nav_data[prn][toe]["i_dot"]
    Omega0 = nav_data[prn][toe]["Omega0"]
    Omega_dot = nav_data[prn][toe]["Omega_dot"]

    _, toe = calc_gps_time(toe)
    _, te = calc_gps_time(te)

    tk = calc_tk(te, toe)


    Mk = M0 + ((np.sqrt(mu) / np.sqrt(a**3)) + dn) * tk
    Ek = solve_kepler(Mk, e)

    Mk = np.mod(Mk, 2 * np.pi)
    Ek = np.mod(Ek, 2 * np.pi)

    teta_k = np.arctan2((np.sin(Ek) * np.sqrt(1 - e**2)), (np.cos(Ek) - e))

    uk = (
        omega
        + teta_k
        + Cuc * np.cos(2 * (omega + teta_k))
        + Cus * np.sin(2 * (omega + teta_k))
    )
    rk = (
        a * (1 - e * np.cos(Ek))
        + Crc * np.cos(2 * (omega + teta_k))
        + Crs * np.sin(2 * (omega + teta_k))
    )
    ik = (
        i0
        + i_dot * tk
        + Cic * np.cos(2 * (omega + teta_k))
        + Cis * np.sin(2 * (omega + teta_k))
    )

    landa_k = Omega0 + (Omega_dot - omega_e) * tk + omega_e * toe
    landa_k = np.mod(landa_k, 2 * np.pi)  ##############

    Omega_k = Omega0 + Omega_dot * tk

    xp = rk * np.cos(uk)
    yp = rk * np.sin(uk)

    Xecef = xp * np.cos(landa_k) - yp * np.cos(ik) * np.sin(landa_k)
    Yecef = xp * np.sin(landa_k) + yp * np.cos(ik) * np.cos(landa_k)
    Zecef = yp * np.sin(ik)

    Xeci = xp * np.cos(Omega_k) - yp * np.cos(ik) * np.sin(Omega_k)
    Yeci = xp * np.sin(Omega_k) + yp * np.cos(ik) * np.cos(Omega_k)
    Zeci = yp * np.sin(ik)

    # Xecef, Yecef, Zecef = eci_to_ecef(Xeci, Yeci, Zeci, tk)
    # r_eci = np.array([Xeci, Yeci, Zeci])

    # r_ecef_converted = eci_to_ecef_vector(r_eci, tk)

    # Xecef, Yecef, Zecef = r_ecef_converted[0], r_ecef_converted[1], r_ecef_converted[2]
    # # print(r_ecef_converted)

    # Xecef_list.append(Xecef)
    # Yecef_list.append(Yecef)
    # Zecef_list.append(Zecef)

    Xecef_list.append(Xecef)
    Yecef_list.append(Yecef)
    Zecef_list.append(Zecef)

    Xeci_list.append(Xeci)
    Yeci_list.append(Yeci)
    Zeci_list.append(Zeci)

    t.append(time() - t1)


r_eci = np.array([Xeci_list, Yeci_list, Zeci_list])
r_ecef = np.array([Xecef_list, Yecef_list, Zecef_list])

print( f'total time: {np.sum(t)}' )

total time: 0.02141261100769043


In [329]:
fig = go.Figure()

# مسیر ECI
fig.add_trace(go.Scatter3d(
    x=r_eci[0], y=r_eci[1], z=r_eci[2],
    mode='lines',
    name='ECI',
    # line=dict(color='blue')
))

# مسیر ECEF
fig.add_trace(go.Scatter3d(
    x=r_ecef[0], y=r_ecef[1], z=r_ecef[2],
    mode='lines',
    name='ECEF',
    # line=dict(color='green')
))

# مرکز زمین (نقطه 0,0,0)
fig.add_trace(go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode='markers',
    marker=dict(size=8, color='red'),
    name='Earth center'
))

# تنظیمات شکل و اندازه
fig.update_layout(
    width=1000,
    height=800,
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)'
    ),
    scene_aspectmode='data',
    title='Satellite Orbits in ECI and ECEF frames'
)

fig.show()
